In [1]:
import numpy as np
import pandas as pd

## not sure need matplot/ scipy/ openpyxl for this - just here because...
# import matplotlib.pyplot as plt
# import scipy as sp
# import openpyxl

## these for scraping pdf's
#import tabula as tb
#import camelot


In [4]:
# define add to dictionary function
def add_allocations(anaesthetist, specialty, allocations):
    if pd.isna(anaesthetist):
        return allocations
    
    anaesthetist = anaesthetist.title().lstrip().rstrip()

    if anaesthetist in allocations.keys():
        allocations[anaesthetist].append(specialty)
    else:
        allocations[anaesthetist] = [specialty]
    return allocations


#extract and work with MOT part of map first
MAP_MOT = pd.read_excel('/home/christopher/Downloads/Theatre Daily map as excel sheet.xlsx', sheet_name = 0, header = 1, index_col = 1, nrows = 9, engine = "openpyxl") #? drop columns 0,1/ 
allocations = dict()

#working with morning lists 
# get list of theaters and drop unnamed (empty) columns
theatres = [x for x in list(MAP_MOT) if 'Unnamed' not in x]
for theatre in theatres:
    specialty_am = MAP_MOT[theatre]['Specialty'][0]
    anaesthetist_am = MAP_MOT[theatre]['Anaesthetist'][0]
    allocations = add_allocations(anaesthetist_am, specialty_am, allocations)

    specialty_pm = MAP_MOT[theatre]['Specialty'][1]
    anaesthetist_pm = MAP_MOT[theatre]['Anaesthetist'][1]
    allocations = add_allocations(anaesthetist_pm, specialty_pm, allocations)

# extract and work with off-floor (lower) portion of map (otf -> 'off_the_floor')
MAP_OTF = pd.read_excel('/home/christopher/Downloads/Theatre Daily map as excel sheet.xlsx', sheet_name = 0, header = 1, index_col = 1, skiprows = 10, nrows = 25, engine = "openpyxl") #? drop columns 0,1/ 
areas = [x for x in list(MAP_OTF)[0:8] if 'Unnamed' not in x]
for area in areas:
    specialty = area
    anaesthetist_am = MAP_OTF[area]['AM'][0]
    allocations = add_allocations(anaesthetist_am, specialty, allocations)

    anaesthetist_pm = MAP_OTF[area]['PM'][0]
    allocations = add_allocations(anaesthetist_pm, specialty, allocations)

###    # Will need correcting when spelling errors in map corrected. ###
MAP_OTF_2 = pd.read_excel('/home/christopher/Downloads/Theatre Daily map as excel sheet.xlsx', sheet_name = 0, header = 1, index_col = 1, skiprows = 14, nrows = 3, engine = "openpyxl") #? drop columns 0,1/ 
allocations = add_allocations(MAP_OTF_2['ECT']['Anaesthetist'], 'ECT', allocations)
allocations = add_allocations(MAP_OTF_2['Echo']['Anaesthetist'], 'Echo', allocations)
allocations = add_allocations(MAP_OTF_2['EP']['Anaesthetist'], 'EP', allocations)
allocations = add_allocations(MAP_OTF_2['Pain']['Anaesthetist'], 'Pain', allocations)

MAP_OTF_3 = pd.read_excel('/home/christopher/Downloads/Theatre Daily map as excel sheet.xlsx', sheet_name = 0, header = 1, index_col = 1, skiprows = 18, nrows = 3, na_values=["NA"], engine = "openpyxl") #? drop columns 0,1/ 
clinical_support = list(MAP_OTF_3.loc['AM'][1:8].dropna()) + list(MAP_OTF_3.loc['PM'][1:8].dropna())

MAP_OTF_4 = pd.read_excel('/home/christopher/Downloads/Theatre Daily map as excel sheet.xlsx', sheet_name = 0, header = 1, index_col = 1, skiprows = 25, nrows = 3, na_values=["NA"], engine = "openpyxl") #? drop columns 0,1/ 
meetings = list(MAP_OTF_4.loc['AM'][1:7].dropna()) + list(MAP_OTF_4.loc['PM'][1:7].dropna())

for anaesthetist in clinical_support:
    allocations = add_allocations(anaesthetist, 'Clinical Support', allocations)
for anaesthetist in meetings:
    allocations = add_allocations(anaesthetist, 'Meetings', allocations)

allocations = pd.DataFrame.from_dict(allocations, orient='index')
allocations.loc['Margale'].value_counts()
allocations

,0,1,2
Kunju,Thoracics,Thoracics,None
Liew,Gen Surg,Gen Surg,None
Postle,Cardiac,Clinical Support,None
Cam Collard,Cardiac,Urology,None
Natani,Tavi,Lead Removal,None
Beuth,Cardiac,Cardiac,None
Howarth,Cardiac,Cardiac,None
Marudachalam,NOF,Ortho,None
Stanton,Ortho,Ortho,None
Larsen,Ortho,Ortho,None
